# 🧠 Manage Your Local Jupyter Server from a Notebook

This notebook provides full control over your local Jupyter server directly from within a Jupyter/Colab/VS Code notebook. You can:

- ✅ Start, stop, restart the server
- 🔍 Check status
- 🌐 Open the local server UI
- 🛠️ Automatically install required and optional packages using a shell script

This is ideal when:
- You're working across Colab and local environments
- You want full reproducibility for AI/ML workflows
- You need an interactive, scriptable environment setup process

---

📂 The shell script it controls is located in the same project directory: `jupyter_server.sh`.

🚨 All critical actions prompt for confirmation unless you override with CLI flags like `--auto-all`.

---


## Server management helper markdown
## 🛠️ Server Management Helper Function

This helper function, `run_server_command(...)`, allows this notebook to call the `jupyter_server.sh` shell script from Python.

Each call:
- Displays which command is being run (`start`, `stop`, `restart`, `status`)
- Prompts you for confirmation before executing (safe by default)
- Shows success/failure messages inline

The script itself supports optional automation flags such as:
- `--auto-all`: install everything
- `--auto-none`: skip all optional installs
- `--auto=gpu,rag`: selective install
- `--prompt`: interactive prompt during setup


In [1]:
# 🛠️ Server Management Helpers (with output)

import os
import subprocess
from IPython.display import display, HTML

# Assume notebook is opened from project root
PROJECT_DIR = os.getcwd()
SCRIPT_PATH = os.path.join(PROJECT_DIR, "jupyter_server.sh")

def run_server_command(command):
    """
    Run a command like start, stop, restart, status
    by calling the shell script directly.
    """
    if not os.path.exists(SCRIPT_PATH):
        print("❌ jupyter_server.sh script not found. Cannot proceed.")
        return
    try:
        result = subprocess.run(["bash", SCRIPT_PATH, command], capture_output=True, text=True)
        if result.stdout:
            print(result.stdout)
        if result.stderr:
            print("⚠️ stderr:")
            print(result.stderr)
    except Exception as e:
        print(f"❌ Error running server command: {e}")

print("✅ Server management helpers loaded successfully.")


✅ Server management helpers loaded successfully.


### ▶️ Start Jupyter Notebook Server

This cell starts the local Jupyter Notebook server in a background process and records its PID.

**What it does:**
- Activates the virtual environment
- Launches Jupyter Notebook on `localhost:8888`
- Writes the server's process ID (PID) to a `.jupyter_server.pid` file

You only need to run this if:
- The server isn't already running
- You want to connect locally or from Colab to your machine

> 💡 To verify it started, check the terminal output and access [http://localhost:8888](http://localhost:8888) in your browser.

---


## 🚀 Start Local Jupyter Server

This cell will start the local Jupyter server using the `jupyter_server.sh` script.

You’ll be asked to confirm before proceeding. If accepted, the server will:
- Start in the background
- Become accessible via [http://localhost:8888](http://localhost:8888/)
- Be visible from Colab (via local runtime) or your browser

To bypass confirmation, you could call:
```python
run_server_command("start", auto_confirm=True)


In [3]:
# 🚀 Start Local Jupyter Server

confirm = input("⚡ Are you sure you want to START the Jupyter server? (yes/no): ")
if confirm.lower() == "yes":
    run_server_command("start")
else:
    print("⛔ Start aborted by user.")


Jupyter server already running (PID 7992).



## 🔄 Restart Local Jupyter Server

This cell stops and then starts the Jupyter server — useful after:
- Installing new packages
- Making config changes
- Clearing memory or hung processes

You’ll get a confirmation prompt before restart.

To run without prompt:
```python
run_server_command("restart", auto_confirm=True)


In [4]:
# 🔄 Restart Local Jupyter Server

confirm = input("🔄 Are you sure you want to RESTART the Jupyter server? (yes/no): ")
if confirm.lower() == "yes":
    run_server_command("restart")
else:
    print("⛔ Restart aborted by user.")


Stopping Jupyter server (PID 7425)...
✅ Server stopped.
Starting Jupyter Notebook server...
✅ Jupyter server started. PID 7992



## 🛑 Stop Local Jupyter Server

Use this cell to stop the currently running Jupyter server process cleanly.

✅ This helps prevent orphaned processes and port binding issues.

Like before:
- You’ll be prompted to confirm.
- To bypass confirmation, use:
```python
run_server_command("stop", auto_confirm=True)


In [6]:
# 🛑 Stop Local Jupyter Server

confirm = input("⚠️ Are you sure you want to STOP the Jupyter server? (yes/no): ")
if confirm.lower() == "yes":
    run_server_command("stop")
else:
    print("⛔ Stop aborted by user.")


⛔ Stop aborted by user.


## 🔍 Check Jupyter Server Status

This cell lets you check if the Jupyter server is currently running.

It’s useful for:
- Verifying whether the server is alive after setup
- Debugging failed connections to `localhost:8888`
- Seeing the PID (process ID) for advanced process control

To skip the prompt and run directly:
```python
run_server_command("status", auto_confirm=True)


In [5]:
# 🔍 Check Jupyter Server Status

run_server_command("status")


✅ Jupyter server is running (PID 7992).



## 🌐 Open Local Jupyter Server Link

This cell provides a clickable link to open your local Jupyter server at:

👉 [http://localhost:8888/](http://localhost:8888/)

You should:
- ✅ Make sure the server is running (`start` or `restart` it if not)
- ✅ Use this link in Colab by going to `Runtime → Connect to local runtime`

💡 If the link fails:
- Use the **status check** to confirm it’s running
- Check for port conflicts or firewall issues


In [7]:
# 🌐 Open Local Jupyter Server Link

from IPython.display import display, HTML

display(HTML("<a href='http://localhost:8888/' target='_blank' style='font-size:20px;'>🚀 Open Local Jupyter Server</a>"))
print("📢 If server is running, click the link above. If it refuses connection, start the server first.")
print("🔄 In Colab: Runtime → Disconnect → Connect to local runtime → Enter http://localhost:8888/")
!jupyter server list

📢 If server is running, click the link above. If it refuses connection, start the server first.
🔄 In Colab: Runtime → Disconnect → Connect to local runtime → Enter http://localhost:8888/
Currently running servers:
http://localhost:8888/?token=009df34aa126640eff1fdcb0ef8da872532e730549d44006 :: /Users/holtharvey/Documents/GitHub/jupyter_server


In [ ]:
import subprocess
import re

# Fetch running server info
server_output = subprocess.getoutput("jupyter server list")

# Search for token pattern
match_token = re.search(r"token=([\w\d]+)", server_output)
match_url = re.search(r"(http://localhost:\d+/\?token=\w+)", server_output)
if match_token and match_url:
    token = match_token.group(1)
    url = match_url.group(1)
    print(f"🌐 URL: {url}")
    print(f"🔐 Token: {token}")
else:
    print("❌ No url or token found. Is the Jupyter server running?")


🔐 URL: http://localhost:8888/?token=009df34aa126640eff1fdcb0ef8da872532e730549d44006
🔐 Token: 009df34aa126640eff1fdcb0ef8da872532e730549d44006


## 🔐 Make Shell Script Executable (Optional in Notebook)
If you plan to run jupyter_server.sh from the terminal, it needs executable permission.

This cell runs:
```bash
chmod +x jupyter_server.sh

In [ ]:
import os

# 📌 Make Shell Script Executable
if os.path.isfile("jupyter_server.sh"):
    !chmod +x jupyter_server.sh
    print("✅ Shell script is now executable.")
else:
    print("❌ jupyter_server.sh not found in current directory.")



✅ Shell script is now executable.


## ⚙️ Setup Environment with Optional Interactive Install

This runs the full setup routine using the `jupyter_server.sh` script with the `--prompt` flag.

You’ll be:
- 🧱 Creating a new Python virtual environment (if not already created)
- 📦 Installing all **base/core** packages listed in `requirements-base.txt`
- ❓ Prompted interactively to choose which **optional modules** (e.g. GPU, RAG, Agents, LLM training) to install

This mode is best when:
- You're doing a **first-time setup**
- You want **control** over what gets installed
- You’re testing or comparing optional environments

If you want full automation instead, use:
```bash
!./jupyter_server.sh setup --auto-all


## 🤖 Install Optional Packages Directly from Notebook (Full Auto Mode)

This cell runs the setup script in **non-interactive mode**, installing:

- ✅ Core base packages (`requirements-base.txt`)
- ✅ All optional groups:
  - GPU support
  - Retrieval-Augmented Generation (RAG)
  - Agentic workflows
  - LLM training libraries

This mode is ideal when:
- You want to fully automate provisioning
- You're setting up new environments on multiple machines
- You trust the entire optional stack to be useful

---

### ⚙️ Script Parameters Overview

You can customize behavior using the following flags:

| Flag | Behavior |
|------|----------|
| `--auto-all` | Installs **all optional packages** with no prompts |
| `--auto-none` | Installs **only base packages**, skips all optional installs |
| `--prompt` | Interactive prompts for each optional group |
| `--auto=gpu,rag` | Installs **only** the listed optional groups (comma-separated list) |

---

⚠️ This full auto mode may take **significantly longer** and use **more memory**.  
If you want selective control instead, run:

```bash
!./jupyter_server.sh setup --prompt



In [10]:
#Setup Environment with Full Auto Install (Base + All Optional)
print("🚀 Running setup with all optional packages included...")
!./jupyter_server.sh setup --auto-all
print("✅ Setup completed. All packages installed.")


🚀 Running setup with all optional packages included...
Virtual environment already exists at /Users/holtharvey/Documents/GitHub/jupyter_server/venv
Installing core requirements (base)...
🧩 Optional installs mode: all
Installing optional group: gpu
Installing optional group: rag
Installing optional group: agents
Installing optional group: llm-training
  Using cached transformers-4.51.3-py3-none-any.whl.metadata (38 kB)
  Using cached sentence_transformers-4.1.0-py3-none-any.whl.metadata (13 kB)
  Using cached peft-0.15.2-py3-none-any.whl.metadata (13 kB)
  Using cached trl-0.17.0-py3-none-any.whl.metadata (12 kB)
  Using cached diffusers-0.33.1-py3-none-any.whl.metadata (19 kB)
  Using cached accelerate-1.6.0-py3-none-any.whl.metadata (19 kB)
  Using cached bitsandbytes-0.42.0-py3-none-any.whl.metadata (9.9 kB)
  Using cached flash_attn-2.7.4.post1.tar.gz (6.0 MB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run 

## 🧭 Environment Monitoring and Version Verification

This cell checks your local environment configuration to ensure compatibility and hardware readiness. It provides:

- 🐍 **Python version** and executable path
- 🧠 **Platform** and **CPU architecture**
- 🔍 **PyTorch**: Installed status and CUDA GPU detection
- 🧪 **TensorFlow**: Installed status and GPU device info
- 🧾 **`watermark`**: Summarized snapshot of all package versions and machine metadata (if extension is available)

> ℹ️ If a module is missing (e.g., `torch`, `tensorflow`, or `watermark`), the script will skip that section gracefully. No crashes.

This step helps **validate installation success**, especially for optional GPU libraries. You can run this cell **at any time** to verify system status after changing packages or re-launching your environment.


In [ ]:
# 📌 Environment Monitoring and Versions Display (Enhanced)
try:
    %load_ext watermark
except:
    print("⚠️ 'watermark' is not installed. Skipping detailed environment printout.")

import sys
import platform

print("🔍 Python Environment Details")
print(f"✅ Python Version     : {platform.python_version()}")
print(f"✅ Platform           : {platform.platform()}")
print(f"✅ Processor          : {platform.processor()}")
import psutil

ram_bytes = psutil.virtual_memory().total
ram_gb = ram_bytes / (1024 ** 3)
print(f"✅ RAM                : {ram_gb:.2f} GB")
print(f"✅ Interpreter Path   : {sys.executable}")

print("\n🔧 Checking PyTorch GPU Support...")
try:
    import torch
    print(f"✅ PyTorch Installed  : Yes")
    print(f"✅ CUDA Available     : {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"🖥️  GPU Name          : {torch.cuda.get_device_name(0)}")
except ImportError:
    print("ℹ️ PyTorch not installed.")

print("\n🔧 Checking TensorFlow GPU Support...")
try:
    import tensorflow as tf
    print(f"✅ TensorFlow Installed: Yes")
    gpu_devices = tf.config.list_physical_devices('GPU')
    if gpu_devices:
        for i, device in enumerate(gpu_devices):
            print(f"🖥️  GPU Device {i}       : {device.name}")
    else:
        print("❌ No GPU devices found.")
except ImportError:
    print("ℹ️ TensorFlow not installed.")

print("\n🧾 Executing watermark (if available)...")
try:
    %watermark --iversions --python --cuda --machine --githash
except:
    print("ℹ️ 'watermark' output skipped (module missing or error).")

print("\n✅ Environment check complete.")


🔍 Python Environment Details
✅ Python Version     : 3.11.8
✅ Platform           : macOS-15.3.2-arm64-arm-64bit
✅ Processor          : arm
✅ Interpreter Path   : /Users/holtharvey/Documents/GitHub/jupyter_server/venv/bin/python

🔧 Checking PyTorch GPU Support...
✅ PyTorch Installed  : Yes
✅ CUDA Available     : False

🔧 Checking TensorFlow GPU Support...
✅ TensorFlow Installed: Yes
❌ No GPU devices found.

🧾 Executing watermark (if available)...
ℹ️ 'watermark' output skipped (module missing or error).

✅ Environment check complete.
